In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.linear_model import LinearRegression

from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.graphics.tsaplots import plot_pacf

# Set figure size to (14,6)
plt.rcParams['figure.figsize'] = (14,6)

# 7.2. Autoregressive Model (AR)

> Autoregression is a time series model that uses observations from previous time steps as input to a regression equation to predict the value at the next time step.

* It is a very simple idea that can result in accurate forecasts on a range of time series problems.
* This technique can be used on time series where input variables are taken as observations at previous time steps, called lag variables.

In [ ]:
# Plot the data
def plot_data(df, title='Remaining Passenger Number after modelling Trend and Seasonality'):
    '''
    Custom plotting function for plotting the flights dataset
    
    Parameters
    ----------
    df : pd.DataFrame
        The data to plot.
    title : str
        The title of the plot
    ylim : bool
        Whether to fix the minimum value of y; defalut is True
    
    Returns
    -------
    Plots the data
    '''
    df.plot()
    plt.title(title)
    plt.ylabel('# of Passengers in 1000')
    plt.show()

## Recap: What we did until now?

---
- Clean data:
     - Clean NA's
     - Format temperature unit multiply (0.1)
     - Skip header
     - Rename and clean columns names
- Built baseline model
- Date to datetime and set to index
- Extract month and year
- Create timestep(trend)
- Create dummies(seasonality months)
- Linear Regression for trend_seasonal

- Get a remainder-get a model that descibe remainder.
    

------

In [11]:
# Load the data
flights = pd.read_csv('flights_train.csv', parse_dates=[0]).set_index('date')
flights.head()

FileNotFoundError: [Errno 2] No such file or directory: 'flights_train.csv'

In [ ]:
# Create a timestep feature for trend
flights['timestep'] = range(len(flights))
flights

In [ ]:
# Create seasonal dummies
seasonal_dummies = pd.get_dummies(flights.index.month,
                                  prefix='month',
                                  drop_first=True).set_index(flights.index)

flights = flights.join(seasonal_dummies)

flights.head()

In [ ]:
# Create a linear regression model
X_flights = flights.drop(columns=['passengers'])
y_flights = flights.passengers

m_flights = LinearRegression()
m_flights.fit(X_flights, y_flights)

flights['trend_seasonal'] = m_flights.predict(X_flights)

In [ ]:
# Plot the model against the real data
plot_data(flights[['passengers', 'trend_seasonal']], title='Passenger numbers vs. predictions')

# WARM-UP
----

## 1) Load the data - IMPORTANT: flights_remainder.csv!

In [ ]:
df = pd.read_csv('./data/flights_remainder.csv', index_col=0, parse_dates=True)
df.head()

In [ ]:
plot_data(df['remainder'])

In [ ]:
#Remainder is not a random walk.
#In real life everything has an error, there is some randomness inside.

## 2) Create a time-lagged input feature: lag1

A lag is an interval of time between two related phenomena. The two related phenomena in this case are the remainder today and the remainder yesterday.

In [ ]:
df['lag1'] = df['remainder'].shift(1)

In [ ]:
df.tail()

In [ ]:
# To predict the temperature of tomorrow we use the temperature of today.

## 3)  Calculate the correlation coefficient between the time-lagged and the original remainder


In [ ]:
df[['remainder']].corrwith(df['lag1'])

In [ ]:
correlation = round(df.corr(), 2)

In [ ]:
sns.heatmap(correlation, annot=True)

#### Intuitively

- It might make sense to look at the value from the last period to predict the value of this period. After taking trend and seasonality into account, a positive remainder today hints towards a positive remainder tomorrow. If today more passengers were flying than we would have expected based on trend and seasonality, then we have reason to believe that the number of passengers flying tomorrow (or next month) will also be higher than predicted by trend and seasonality.

- In the case of temperature, this would mean: if it is warmer than usual (by usual we tend to mean after taking into account seasonality and trend)today, it is likely to be warmer than usual tomorrow.

## 4) Create a scatterplot with df['lag1'] on the x-axis and df['remainder'] on the y-axis


In [ ]:
# Create a scatterplot
sns.scatterplot(x='lag1', y='remainder', data=df)
plt.title('Relationship of Lag and Remainder')
plt.show()

----

# AR - Autoregressive Model

Now we are ready to talk about the autoregressive model. In the autoregressive model we model the future value of a variable by looking at the present (and maybe past) value of the same variable. In our case the variable will be the **remainder**.

$$
y_{t+1} = w_0 + \sum_{j=0}^{P-1}w_{j}y_{t-j} + \epsilon_t
$$

where $t$ is the current timestep, $w_0$ is the bias/intercept, the w’s are the weights of the model and $\epsilon$ is some iid (e.g. Gaussian) noise. **P** is the number of lags to use for the model. It is a hyperparameter that we have to choose.

Mean and the variance of the error are constant and time invariant.

**AR(1) - Model**

$$
\hat{y_{t+1}} = w_0 + w_1 * y_{t}
$$

or in case of our remainder

$$
\hat{remainder_{t+1}} = w_0 + w_1 * remainder_{t}
$$

In an AR(1) model we model the value in $t+1$ as a linear function of the value in $t$.

**AR(2) - Model**

$$
\hat{y_{t+1}} = w_0 + w_1 * y_{t} + w_2 * y_{t-1}
$$

## 5) Run an Autoregressive Model (Linear Regression) of lag1 on the remainder

In [ ]:
# Drop missing values
df.dropna(inplace=True)
df.head()

In [ ]:
# Assign X and y
X = df[['lag1']]
y = df['remainder']

In [ ]:
# Create and fit the model
ar_model = LinearRegression()
ar_model.fit(X, y)

In [ ]:
# Create predictions
df['ar_predictions'] = ar_model.predict(X)

In [ ]:
# Plot the original remainder and the prediction
plot_data(df[['remainder', 'ar_predictions']])

In [ ]:
# Inspect the residual after modelling the remainder
plt.plot(df.remainder - df.ar_predictions)

## 6) Should we add lag2?



In [ ]:
# Create a second lag
df['lag2'] = df.remainder.shift(2)
df.head()

In [ ]:
# Plot the second lag against the remainder
sns.scatterplot(x='lag2', y='remainder', data=df)
plt.title('Remainder and second lag')
plt.show()

Is this relationship meaningful?

In [ ]:
# Inspect correlations
correlations = round(df[['remainder', 'lag1', 'lag2']].corr(), 2)
sns.heatmap(correlations, annot=True)

# * Autocorrelation


- We can use statistical measures to calculate the correlation between the output variable and values at previous time steps at various different lags. The stronger the correlation between the output variable and a specific lagged variable, the more weight that autoregression model can put on that variable when modeling.


- Again, because the correlation is calculated between the variable and itself at previous time steps, it is called an autocorrelation. 

##  Partial Autocorrelation

One tool to find out how many lags we should include into our Autoregressive Model is to plot the Partial Autocorrelation between different lags.

#### What is the Partial Autocorrelation?

$$
\delta_h = Corr(y_t, y_{t-h}|y_{t-1}, ..., y_{t-h+1})
$$

A measure of time dependence is the so-called partial-autocorrelation function (PACF), which is the correlation between $y_t$ and $y_{t−h}$, conditional on the intermediate values (eg. $y_{t-1}$, $y_{t-2}$, ...).

They are basically nothing else than the coefficients in a linear regression if you included h lags.


#### Summary 

*Autocorrelation(ACF)*
Correlation between observation at the current time spot and the observations at previous time spot.

*Partial Auto Correlation(PACF)*
The correlation between observations at two time spots given that we consider both observations are correlated to observations at other
time spots. For example, todays temperature can be correlated to the day before yesterday and yesterday can be also correlated to the day before yesterday. Then PACF of yesterday is the real corrleation between today and yesterday after taking out the influence of the day before yesterday.

In [ ]:
# Statsmodels provides a function called plot_pacf
# plot partial autocorrelation function

from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(df['remainder']);
plt.xlabel('Nr. of lags')
plt.ylabel('Partial Autocorrelation')

>Like the correlation coefficient, the partial correlation coefficient takes on a value in the range from –1 to 1. 

In [ ]:

# first point is a remainder.

**Box-Jenkins-Methodology:**

* [Box-Jenkins-Method](https://en.wikipedia.org/wiki/Box–Jenkins_method)


This methodology makes a statement about how to choose lags.

We will choose all lags until the first partial autocorrelation value ends up within the confidence interval. In this case we will have 1 lag.

In [ ]:
df['residuals_ar1'] = df['remainder'] - df['ar_predictions']
df.head()

In [ ]:
df[['lag2']].corrwith(df['residuals_ar1'])

In [ ]:
# Plot the second lag against the remainder
sns.scatterplot(x='lag2', y='residuals_ar1', data=df)
plt.title('Residuals of AR(1) and second lag')
plt.show()

## 6.2) Introduce ar_select_order

In [ ]:
# Let ar_select_order select the number of lags for the remainder
selected_order = ar_select_order(df['remainder'], maxlag=12)

# ar_select_order iteratively fits and AR(1) model, then an AR(2) model, ..., finally an AR(maxlag) and returns the one that performs best
# The decision which model works best is based on an evaluation metric called AIC (Akaike Information Criterion)

In [ ]:
# Inspect the number of lags chosen
selected_order.ar_lags # tells us to use 1 lag

# What now?  👀

- We completely focused on modelling the remainder separate from everything else.
- In the end we are still interested in the absolut number of passengers (or temperature), not in the remainder itself

Next step: Add the lag of the original variable (or the remainder) as a feature to your DataFrame and create a new model with the features timestep, seasonal_dummies and the appropriate number of lags

In [ ]:
flights

We analyzed that it makes most sense to model the remainder by an AR(1) model. How do we include this knowledge into our modelling approach?

In [ ]:
# Create a new column with the lag1 of passengers
flights['lag1_passengers'] = flights.passengers.shift(1)
flights.dropna(inplace=True)
flights.head()

In [ ]:
# Train a "full" model including the autoregressive part
X_full = flights.drop(columns=['passengers', 'trend_seasonal'])
y_full = flights.passengers

m_full = LinearRegression()
m_full.fit(X_full, y_full)

In [ ]:
# Make predictions using the full model
flights['pred_full'] = m_full.predict(X_full)

In [ ]:
plot_data(flights[['passengers', 'trend_seasonal', 'pred_full']], title='Compare trend-seasonal model to the full model including the AR part')

**The analysis about how many lags to include into the model has to be done using the remainder**. This is because the remainder is stripped of the influence of trend and seasonality. But in order to build the actual model, we can use the lag of the actual passenger data.

In [ ]:
# Build the same model using statsmodels
statsmodels_ar = AutoReg(flights['passengers'], lags=1, trend='ct', seasonal=True).fit()

In [ ]:
plt.plot(statsmodels_ar.predict())
plt.plot(flights['pred_full'])